In [ ]:
# 앞서 배운 메모리 클래스 중 하나를 사용하는 메모리로 LCEL 체인을 구현합니다.
# 이 체인은 영화 제목을 가져와 영화를 나타내는 세 개의 이모티콘으로 응답해야 합니다. (예: "탑건" -> "🛩️👨‍✈️🔥". "대부" -> "👨‍👨‍👦🔫🍝").
# 항상 세 개의 이모티콘으로 답장하도록 FewShotPromptTemplate 또는 FewShotChatMessagePromptTemplate을 사용하여 체인에 예시를 제공하세요.
# 메모리가 작동하는지 확인하려면 체인에 두 개의 영화에 대해 질문한 다음 다른 셀에서 체인에 먼저 질문한 영화가 무엇인지 알려달라고 요청하세요.

from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler(),]
)

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

class RandomExampleSelector(BaseExampleSelector):

    def __init__(self, examples):
        self.examples = examples
    
    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)]
        

example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

example_selector = RandomExampleSelector(
    examples = examples,

)

prompt = FewShotPromptTemplate(
    example_prompt = example_prompt,
    example_selector = example_selector,
    suffix = "Human: what do you know about {country}?",
    input_variables=["country"],
)
prompt.format(country = "Brazil")

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.memory import ConversationBufferWindowMemory
from langchain.schema.runnable import RunnablePassthrough


llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.8,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=10,
)

examples = [
    {
        "question": "I want a summary of the movie 'Top Gun' only written by emojis.",
        "answer": "🛩️👨‍✈️🔥",
    },
    {
        "question": "I want a summary of the movie 'The Godfather' only written by emojis.",
        "answer": "👨‍👨‍👦🔫🍝",
    },
    {
        "question": "I want a summary of the movie 'Interstellar' only written by emojis.",
        "answer": "🧑‍🚀🚀⭐",
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{question}"),
        ("ai", "{answer}"),
    ]
)

chat_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a movie lover. Basically, when given question about a movie, you reply with exactly 3 emoji that represents the movie well. But when asked the movie that you answered before, you must reply with the same 3 emoji that you answered before. Also, when human asks you about movies that you remember, you reply every movie title that you remember.",
        ),
        # example을 MessagesPlaceholder보다 먼저 주면 AI가 history보다 example을 더 중요시에서 example을 토대로 답할 수도 있다
        MessagesPlaceholder(variable_name="history"),
        chat_prompt,
        (
            "human",
            "{question}",
        ),
    ]
)


def save(input, output):
    memory.save_context(
        {"input": input},
        {"output": output},
    )


save("I want a summary of the movie 'Top Gun' only written by emojis.", "🛩️👨‍✈️🔥")
save(
    "I want a summary of the movie 'The Godfather' only written by emojis.",
    "👨‍👨‍👦🔫🍝",
)
save("I want a summary of the movie 'Interstellar' only written by emojis.", "🧑‍🚀🚀⭐")


def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm


def invoke_by_movie(movie):
    question = f"I want a summary of the movie '{movie}' only written by emojis."
    result = chain.invoke({"question": question})
    save(question, result.content)


def invoke_by_question(question):
    result = chain.invoke({"question": question})
    save(question, result.content)

In [ ]:
invoke_by_movie("Avengers: Endgame")

In [ ]:
invoke_by_movie("Dune")